In [5]:
import math
import numpy as np
import random
from tqdm.notebook import tqdm
import nashpy as nash
import matplotlib.pyplot as plt
import json


class Machine:
    
    def __init__(self, vulnerable, percentValue, penetrate, protectionLevel, possibilityProtection):
        #Уязвима или нет - bool
        self.vulnerable = vulnerable
        #Значение в сети - %
        self.percentValue = percentValue
        #Взломана или нет -bool
        self.penetrate = penetrate
        #Защита - E' * percentValue
        self.protectionLevel = protectionLevel
        #Возможно ли ее защитить - bool
        self.possibilityProtection = possibilityProtection
        
    def changeVulnerable(self, vulnerable):
        self.vulnerable = vulnerable
    
    def changeCost(self, percentValue):
        self.percentValue = float(percentValue)
        
    def changePenetrate(self, penetrate):
        self.penetrate = penetrate
    
    def changeProtectionLevel(self, protectionLevel):
        self.protectionLevel = float(protectionLevel)
    
    def changePossibilityProtection(self, possibilityProtection):
        self.possibilityProtection = possibilityProtection
        
    def pr(self):
        print(self.vulnerable, self.percentValue, self.penetrate)

        
def printAllvalues(machineValues, vulnerableMachineValues, importantMachineValues):
    print('machine values = ', machineValues, 'vulnerable machine values = ', vulnerableMachineValues, 'important machine values =', importantMachineValues)
#получаем кол-во машин
def GetMachineValues():
    machineValues = random.randint(4, 15)
    return machineValues
#получаем кол-во уязвимых машин 
def GetVulnerableMachineValues(machineValues):
    vulnerableMachineValues = random.randint(1, machineValues)
    return vulnerableMachineValues
#получаем номер главной машины
def GetImportantMachineValues(machineValues):
    importantMachineValues = random.randint(1, machineValues)
    return importantMachineValues
#получаем процент, который занимает главная машина 
def GetPercentValue():
    percentValue = random.randint(30, 65)
    return percentValue

def GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues):
    machineArray = []
    percentValue = GetPercentValue()
    reversePercent = ((100 - percentValue) / (machineValues - 1)) / 100
    reversePercent = ("%.2f" % reversePercent)
    for i in range(machineValues):
        if i == (importantMachineValues - 1):
            tmp = Machine(False, float(percentValue / 100), False, float(0.0), False)
        else:
            tmp = Machine(False, float(reversePercent), False, float(0.0), False)
        machineArray.append(tmp)
    #здесь баг - может 2 и более раз сгенерироваться
    #одинаковое число и поэтому кол-во машин будет неточным
    for i in range(0, vulnerableMachineValues):
        j = random.randint(0, machineValues - 1)
        machineArray[j].changeVulnerable(True)
    return machineArray
#возвращает число уязвимых машин, так как из-за бага
#прошлое число уязвимых != новому
def GetVulnerableMachine(machineValues, machineArray):
    vulnerableMachine = 0
    for i in range(machineValues):
        if machineArray[i].vulnerable == True:
            vulnerableMachine += 1
    return vulnerableMachine
#это все возможные разрывы функции стоимости на данном промежутке 1-10
def CheckCost(a):
    BreakFunc = {
        '1':'1',
        '2':'0.5',
        '3':'0.333',
        '4':'0.25',
        '5':'0.2',
        '6':'0.166',
        '7':'0.142',
        '8':'0.125',
        '9':'0.111',
        '10':'0.1'
    }
    return BreakFunc[a]

def CostFunction(s, a, Ei):
    try:
        return -(s * Ei * Ei)/(a * Ei - 1)
    except ZeroDivisionError:
        return 0
    #С условием в распределении уровней защиты в стратегиях - не должно быть деления на 0
    
def UtilityDefender(s, a, Ei, machineArray):
    tmp = 0
    percent = len(machineArray) / 100
    l = 0
    for i in range(len(machineArray)):
        if machineArray[i].penetrate == True:
            l += percent
            tmp += CostFunction(s, a, machineArray[i].protectionLevel)
    return - l - tmp

def UtilityAttacker(machineArray):
    percent = len(machineArray) / 100
    l = 0
    for i in range(len(machineArray)):
        if machineArray[i].penetrate == True:
            l += percent
    return l
    
def AttackerStrategyProportional(machineArray):
    attackerProportionalEffort = []
    for i in range(len(machineArray)):
        tmp = machineArray[i].percentValue
        attackerProportionalEffort.append(tmp)
    return attackerProportionalEffort

def AttackerStrategyProportionalRedistribution(machineArray):
    attackerProportionalEffort = []
    for i in range(len(machineArray)):
        if machineArray[i].penetrate == False:
            tmp = machineArray[i].percentValue
            attackerProportionalEffort.append(tmp)
        else:
            attackerProportionalEffort.append(0)
    return attackerProportionalEffort
        
def AttackerStrategyHigh():
    attackerHighEffort = 1
    return attackerHighEffort

def AttackerStrategyHighRedistribution():
    attackerHighEffort = 1
    return attackerHighEffort
        
def DefenderStrategyNothing():
    return None
    
def DefenderStrategyProportional(machineArray, Ei, a):
    for i in range(len(machineArray)):
        if float(CheckCost(str(a))) < machineArray[i].percentValue * Ei:
            machineArray[i].changePossibilityProtection(False)
            machineArray[i].changeProtectionLevel(0)
        else:
            machineArray[i].changePossibilityProtection(True)
            machineArray[i].changeProtectionLevel("%.2f" % (machineArray[i].percentValue * Ei))
    
def DefenderStrategyHigh(Ei, importantMachineValues, machineArray, a):
    for i in range(len(machineArray)):
        machineArray[i].changePossibilityProtection(False)
        machineArray[i].changeProtectionLevel(0)
    if float(CheckCost(str(a))) > Ei:
        machineArray[importantMachineValues - 1].changePossibilityProtection(True)
        machineArray[importantMachineValues - 1].changeProtectionLevel(Ei)       
#P-атакующий H-защитник
def ComparisonStrategyPH(machineArray, importantMachineValues, Ei):
    attackerProportionalEffort = AttackerStrategyProportional(machineArray)
    DefenderStrategyHigh(Ei, importantMachineValues, machineArray, a)
    count = 0
    for i in range(len(machineArray)):
        if machineArray[i].vulnerable == True:
            if attackerProportionalEffort[i] > machineArray[i].protectionLevel:
                machineArray[i].changePenetrate(True)
                count += 1
    if count > 0:
        return True
    else:
        return False
#RP - Redistribution Proportional атакующий H - защитник        
def ComparisonStrategyRPH(machineArray, importantMachineValues, Ei):
    DefenderStrategyHigh(Ei, importantMachineValues, machineArray, a) 
    attackerRedistributionProportionalEffort = AttackerStrategyProportionalRedistribution(machineArray)
    count = 0
    for i in range(len(machineArray)):
        if machineArray[i].vulnerable == True and machineArray[i].penetrate == False:
            if attackerRedistributionProportionalEffort[i] > machineArray[i].protectionLevel:
                machineArray[i].changePenetrate(True)
                count += 1
    if count > 0:
        return True
    else:
        return False 
#P-атакующий P-защитник
def ComparisonStrategyPP(machineArray, Ei, a):
    attackerProportionalEffort = AttackerStrategyProportional(machineArray)
    DefenderStrategyProportional(machineArray, Ei, a)
    count = 0
    for i in range(len(machineArray)):
        if machineArray[i].vulnerable == True:
            if attackerProportionalEffort[i] > machineArray[i].protectionLevel:
                machineArray[i].changePenetrate(True)
                count += 1
    if count > 0:
        return True
    else:
        return False
#RP - атакующий P - защитник
def ComparisonStrategyRPP(machineArray, Ei, a):
    DefenderStrategyProportional(machineArray, Ei, a)
    attackerRedistributionProportionalEffort =  AttackerStrategyProportionalRedistribution(machineArray)
    count = 0
    for i in range(len(machineArray)):
        if machineArray[i].vulnerable == True and machineArray[i].penetrate == False:
            if attackerRedistributionProportionalEffort[i] > machineArray[i].protectionLevel:
                machineArray[i].changePenetrate(True)
                count += 1
    if count > 0:
        return True
    else:
        return False
#P - атакующий N - защитник
def ComparisonStrategyPN(machineArray):
    attackerProportionalEffort = AttackerStrategyProportional(machineArray)
    count = 0
    for i in range(len(machineArray)):
        if machineArray[i].vulnerable == True:
            if attackerProportionalEffort[i] > machineArray[i].protectionLevel:
                machineArray[i].changePenetrate(True)
                count += 1
    if count > 0:
        return True
    else:
        return False
#RP - атакующий N - защитник
def ComparisonStrategyRPN(machineArray):
    attackerRedistributionProportionalEffort =  AttackerStrategyProportionalRedistribution(machineArray)
    count = 0
    for i in range(len(machineArray)):
        if machineArray[i].vulnerable == True and machineArray[i].penetrate == False:
            if attackerRedistributionProportionalEffort[i] > machineArray[i].protectionLevel:
                machineArray[i].changePenetrate(True)
                count += 1
    if count > 0:
        return True
    else:
        return False
#H-атакующий P-защитник
def ComparisonStrategyHP(machineArray, Ei, importantMachineValues, a):
    attackerHighEffort = AttackerStrategyHigh()
    DefenderStrategyProportional(machineArray, Ei, a)
    if attackerHighEffort > machineArray[importantMachineValues - 1].protectionLevel:
        if machineArray[importantMachineValues - 1].vulnerable == True:
            machineArray[importantMachineValues - 1].changePenetrate(True)
            return True
        else:
            return False
#RH-атакующий P-защитник
def ComparisonStrategyRHP(machineArray, Ei, importantMachineValues, a):
    attackerHighRedistributionEffort = AttackerStrategyHighRedistribution()
    DefenderStrategyProportional(machineArray, Ei, a)
    if machineArray[importantMachineValues - 1].vulnerable == True and machineArray[importantMachineValues - 1].penetrate == False:
        if attackerHighRedistributionEffort > machineArray[importantMachineValues - 1].protectionLevel:
            machineArray[importantMachineValues - 1].changePenetrate(True)
            return True
        else:
            return False
#H-атакующий H-защитник
def ComparsionStrategyHH(Ei, importantMachineValues, machineArray, a):
    attackerHighEffort = AttackerStrategyHigh()
    DefenderStrategyHigh(Ei, importantMachineValues, machineArray, a)
    if attackerHighEffort > machineArray[importantMachineValues - 1].protectionLevel:
        if machineArray[importantMachineValues - 1].vulnerable == True:
            machineArray[importantMachineValues - 1].changePenetrate(True)
            return True
        else:
            return False
#RH-атакующий H-защитник
def ComparsionStrategyRHH(Ei, importantMachineValues, machineArray, a):
    attackerHighRedistributionEffort = AttackerStrategyHighRedistribution()
    DefenderStrategyHigh(Ei, importantMachineValues, machineArray, a)
    if attackerHighRedistributionEffort > machineArray[importantMachineValues - 1].protectionLevel:
        if machineArray[importantMachineValues - 1].vulnerable == True and machineArray[importantMachineValues - 1].penetrate == False:
            machineArray[importantMachineValues - 1].changePenetrate(True)
            return True
        else:
            return False
#H-Атакующий N-защитник
def ComparsionStrategyHN(machineArray, importantMachineValues):
    attackerHighEffort = AttackerStrategyHigh()
    if attackerHighEffort > machineArray[importantMachineValues - 1].percentValue:
        if machineArray[importantMachineValues - 1].vulnerable == True:
            machineArray[importantMachineValues - 1].changePenetrate(True)
            return True
        else:
            return False
#RH-Атакующий N-защитник
def ComparsionStrategyRHN(machineArray, importantMachineValues):
    attackerHighRedistributionEffort = AttackerStrategyHighRedistribution()
    if attackerHighRedistributionEffort > machineArray[importantMachineValues - 1].percentValue:
        if machineArray[importantMachineValues - 1].vulnerable == True and machineArray[importantMachineValues - 1].penetrate == False:
            machineArray[importantMachineValues - 1].changePenetrate(True)
            return True
        else:
            return False
        
def CheckStrategyDefender(b):
    NumberStrategy = {
        '1':'Noting',
        '2':'High',
        '3':'High',
        '4':'High',
        '5':'High',
        '6':'High',
        '7':'High',
        '8':'High',
        '9':'High',
        '10':'High',
        '11':'Proportional',
        '12':'Proportional',
        '13':'Proportional',
        '14':'Proportional',
        '15':'Proportional',
        '16':'Proportional',
        '17':'Proportional',
        '18':'Proportional',
        '19':'Proportional'
    }
    return NumberStrategy[b]
    
#def JsonCheck():
    
    
arrayEqlibriumDefender = []
arrayEqlibriumAttecker = []

for s in (0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9):
    for a in range(1, 11, 1):
        shape = (19,2) #матрица игры
        Attacker = np.zeros(shape=shape)
        Defender = np.zeros(shape=shape)
        #1 - Proportional 2 - High
        for strategyA in (1, 2):
            #1 - High 2 - Proportional 3 - Nothing
            for strategyD in (1, 2, 3):
                for Ei in (0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9):
                    if strategyA == 1 and strategyD == 1: #A -Proportional, D - High
                        j = 0
                        utilityDefender = 0
                        utilityAttacker = 0
                        for j in tqdm(range(50000)):
                            machineValues = GetMachineValues()
                            vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                            importantMachineValues = GetImportantMachineValues(machineValues)
                            machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                            tmp = ComparisonStrategyPH(machineArray, importantMachineValues, Ei)
                            while tmp == True:
                                if tmp == True:
                                    utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                                    utilityAttacker += UtilityAttacker(machineArray)
                                    tmp = ComparisonStrategyRPH(machineArray, importantMachineValues, Ei)
                                else:
                                    utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                        print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
                        print('Utility A =', utilityAttacker, 'utility D =', utilityDefender)
                    elif strategyA == 1 and strategyD == 2:
                        j = 0
                        utilityDefender = 0
                        utilityAttacker = 0
                        for j in tqdm(range(50000)):
                            machineValues = GetMachineValues()
                            vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                            importantMachineValues = GetImportantMachineValues(machineValues)
                            machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                            tmp = ComparisonStrategyPP(machineArray, Ei, a)
                            while tmp == True:
                                if tmp == True:
                                    utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                                    utilityAttacker += UtilityAttacker(machineArray)
                                    tmp = ComparisonStrategyRPP(machineArray, Ei, a)
                                else:
                                    utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                        print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
                        print('Utility A =', utilityAttacker, 'utility D =', utilityDefender)
                    elif strategyA == 1 and strategyD == 3:
                        j = 0
                        utilityDefender = 0
                        utilityAttacker = 0
                        for j in tqdm(range(50000)):
                            machineValues = GetMachineValues()
                            vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                            importantMachineValues = GetImportantMachineValues(machineValues)
                            machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                            tmp = ComparisonStrategyPN(machineArray)
                            while tmp ==True:
                                if tmp == True:
                                    utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                                    utilityAttacker += UtilityAttacker(machineArray)
                                    tmp = ComparisonStrategyRPN(machineArray)
                                else:
                                    utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                                    break
                        print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
                        print('Utility A =', utilityAttacker, 'utility D =', utilityDefender)
                    elif strategyA == 2 and strategyD == 1:
                        j = 0
                        utilityDefender = 0
                        utilityAttacker = 0
                        for j in tqdm(range(50000)):
                            machineValues = GetMachineValues()
                            vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                            importantMachineValues = GetImportantMachineValues(machineValues)
                            machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                            tmp = ComparsionStrategyHH(Ei, importantMachineValues, machineArray, a)
                            while True:
                                if tmp == True:
                                    utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                                    utilityAttacker += UtilityAttacker(machineArray)
                                    tmp = ComparsionStrategyRHH(Ei, importantMachineValues, machineArray, a)
                                else:
                                    utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                                    break
                        print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
                        print('Utility A =', utilityAttacker, 'utility D =', utilityDefender) 
                    elif strategyA == 2 and strategyD == 2:
                        j = 0
                        utilityDefender = 0
                        utilityAttacker = 0
                        for j in tqdm(range(50000)):
                            machineValues = GetMachineValues()
                            vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                            importantMachineValues = GetImportantMachineValues(machineValues)
                            machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                            tmp = ComparisonStrategyHP(machineArray, Ei, importantMachineValues, a)
                            while True:
                                if tmp == True:
                                    utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                                    utilityAttacker += UtilityAttacker(machineArray)
                                    tmp = ComparisonStrategyRHP(machineArray, Ei, importantMachineValues, a)
                                else:
                                    utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                                    break
                        print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
                        print('Utility A =', utilityAttacker, 'utility D =', utilityDefender)
                    elif strategyA == 2 and strategyD == 3:
                        j = 0
                        utilityDefender = 0
                        utilityAttacker = 0
                        for j in tqdm(range(50000)):
                            machineValues = GetMachineValues()
                            vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                            importantMachineValues = GetImportantMachineValues(machineValues)
                            machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                            tmp = ComparsionStrategyHN(machineArray, importantMachineValues)
                            while True:
                                if tmp == True:
                                    utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                                    utilityAttacker += UtilityAttacker(machineArray)
                                    tmp = ComparsionStrategyRHN(machineArray, importantMachineValues)
                                else:
                                    utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                                    break
                        print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
                        print('Utility A =', utilityAttacker, 'utility D =', utilityDefender)
                    if strategyD == 3:
                        Attacker[0, strategyA - 1] = round(utilityAttacker)
                        Defender[0, strategyA - 1] = round(utilityDefender)
                    elif strategyD == 1:
                        index = int(Ei * 10)
                        Attacker[index, strategyA - 1] = round(utilityAttacker)
                        Defender[index, strategyA - 1] = round(utilityDefender)
                    elif strategyD == 2:
                        index = int(Ei * 10) + 9
                        Attacker[index, strategyA - 1] = round(utilityAttacker)
                        Defender[index, strategyA - 1] = round(utilityDefender)
        degenGameTest = False
        game = nash.Game(Defender, Attacker)
        print("Утилиты атакующего -", Attacker)
        print("Утилиты обороняющегося -", Defender)
        try:
            equlibrium = list(game.support_enumeration())[0]
        except IndexError:
            try:
                equlibrium = list(game.support_enumeration(non_degenerate=True))[0]
            except:
                equlibrium = list(game.support_enumeration(non_degenerate=True))
                degenGameTest = True
        print("equlibrium", equlibrium)
        if degenGameTest == False:
            numEqulibriumDefender = np.argmax(equlibrium[0])+1
            numEqulibriumAttacker = np.argmax(equlibrium[1])+1
        else:
            numEqulibriumDefender = equlibrium
            numEqulibriumAttacker = equlibrium
        print("оптимальная стратегия оброняющегося номер ", numEqulibriumDefender, "оптимальная стратегия атакующего номер ", numEqulibriumAttacker)
        arrayEqlibriumDefender.append(numEqulibriumDefender)
        arrayEqlibriumAttecker.append(numEqulibriumAttacker)

print('array Defender - ', '\n', arrayEqlibriumDefender)
print('arrau Attacker - ', '\n', arrayEqlibriumAttecker)

  0%|          | 0/50000 [00:00<?, ?it/s]

for s = 0.1 a = 1 sA =  1 sd = 1 Ei = 0.1
Utility A = 20905.70000000181 utility D = -20929.214444444806


  0%|          | 0/50000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [2]:
#for s in (0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9):
#    for a in range(1, 11, 1):
s = 0.6 
a = 5
shape = (19,2) #матрица игры
Attacker = np.zeros(shape=shape)
Defender = np.zeros(shape=shape)
#1 - Proportional 2 - High
for strategyA in (1, 2):
    #1 - High 2 - Proportional 3 - Nothing
    for strategyD in (1, 2, 3):
        for Ei in (0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9):
            if strategyA == 1 and strategyD == 1: #A -Proportional, D - High
                j = 0
                utilityDefender = 0
                utilityAttacker = 0
                for j in tqdm(range(50000)):
                    machineValues = GetMachineValues()
                    vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                    importantMachineValues = GetImportantMachineValues(machineValues)
                    machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                    tmp = ComparisonStrategyPH(machineArray, importantMachineValues, Ei)
                    while tmp == True:
                        if tmp == True:
                            utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                            utilityAttacker += UtilityAttacker(machineArray)
                            tmp = ComparisonStrategyRPH(machineArray, importantMachineValues, Ei)
                        else:
                            utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
                print('Utility A =', utilityAttacker, 'utility D =', utilityDefender)
            elif strategyA == 1 and strategyD == 2:
                j = 0
                utilityDefender = 0
                utilityAttacker = 0
                for j in tqdm(range(50000)):
                    machineValues = GetMachineValues()
                    vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                    importantMachineValues = GetImportantMachineValues(machineValues)
                    machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                    tmp = ComparisonStrategyPP(machineArray, Ei, a)
                    while tmp == True:
                        if tmp == True:
                            utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                            utilityAttacker += UtilityAttacker(machineArray)
                            tmp = ComparisonStrategyRPP(machineArray, Ei, a)
                        else:
                            utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
                print('Utility A =', utilityAttacker, 'utility D =', utilityDefender)
            elif strategyA == 1 and strategyD == 3:
                j = 0
                utilityDefender = 0
                utilityAttacker = 0
                for j in tqdm(range(50000)):
                    machineValues = GetMachineValues()
                    vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                    importantMachineValues = GetImportantMachineValues(machineValues)
                    machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                    tmp = ComparisonStrategyPN(machineArray)
                    while tmp ==True:
                        if tmp == True:
                            utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                            utilityAttacker += UtilityAttacker(machineArray)
                            tmp = ComparisonStrategyRPN(machineArray)
                        else:
                            utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                            break
                print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
                print('Utility A =', utilityAttacker, 'utility D =', utilityDefender)
            elif strategyA == 2 and strategyD == 1:
                j = 0
                utilityDefender = 0
                utilityAttacker = 0
                for j in tqdm(range(50000)):
                    machineValues = GetMachineValues()
                    vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                    importantMachineValues = GetImportantMachineValues(machineValues)
                    machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                    tmp = ComparsionStrategyHH(Ei, importantMachineValues, machineArray, a)
                    while True:
                        if tmp == True:
                            utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                            utilityAttacker += UtilityAttacker(machineArray)
                            tmp = ComparsionStrategyRHH(Ei, importantMachineValues, machineArray, a)
                        else:
                            utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                            break
                print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
                print('Utility A =', utilityAttacker, 'utility D =', utilityDefender) 
            elif strategyA == 2 and strategyD == 2:
                j = 0
                utilityDefender = 0
                utilityAttacker = 0
                for j in tqdm(range(50000)):
                    machineValues = GetMachineValues()
                    vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                    importantMachineValues = GetImportantMachineValues(machineValues)
                    machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                    tmp = ComparisonStrategyHP(machineArray, Ei, importantMachineValues, a)
                    while True:
                        if tmp == True:
                            utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                            utilityAttacker += UtilityAttacker(machineArray)
                            tmp = ComparisonStrategyRHP(machineArray, Ei, importantMachineValues, a)
                        else:
                            utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                            break
                print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
                print('Utility A =', utilityAttacker, 'utility D =', utilityDefender)
            elif strategyA == 2 and strategyD == 3:
                j = 0
                utilityDefender = 0
                utilityAttacker = 0
                for j in tqdm(range(50000)):
                    machineValues = GetMachineValues()
                    vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
                    importantMachineValues = GetImportantMachineValues(machineValues)
                    machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
                    tmp = ComparsionStrategyHN(machineArray, importantMachineValues)
                    while True:
                        if tmp == True:
                            utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                            utilityAttacker += UtilityAttacker(machineArray)
                            tmp = ComparsionStrategyRHN(machineArray, importantMachineValues)
                        else:
                            utilityDefender += UtilityDefender(s, a, Ei, machineArray)
                            break
                print('for s =', s,'a =', a, 'sA = ', strategyA, 'sd =', strategyD, 'Ei =', Ei)
                print('Utility A =', utilityAttacker, 'utility D =', utilityDefender)
            if strategyD == 3:
                Attacker[0, strategyA - 1] = round(utilityAttacker)
                Defender[0, strategyA - 1] = round(utilityDefender)
            elif strategyD == 1:
                index = int(Ei * 10)
                Attacker[index, strategyA - 1] = round(utilityAttacker)
                Defender[index, strategyA - 1] = round(utilityDefender)
            elif strategyD == 2:
                index = int(Ei * 10) + 9
                Attacker[index, strategyA - 1] = round(utilityAttacker)
                Defender[index, strategyA - 1] = round(utilityDefender)
degenGameTest = False
game = nash.Game(Defender, Attacker)
print("Утилиты атакующего -", Attacker)
print("Утилиты обороняющегося -", Defender)
try:
    equlibrium = list(game.support_enumeration())[0]
except IndexError:
    try:
        equlibrium = list(game.support_enumeration(non_degenerate=True))[0]
    except:
        equlibrium = list(game.support_enumeration(non_degenerate=True))
        degenGameTest = True
print("equlibrium", equlibrium)
if degenGameTest == False:
    numEqulibriumDefender = np.argmax(equlibrium[0])+1
    numEqulibriumAttacker = np.argmax(equlibrium[1])+1
else:
    numEqulibriumDefender = equlibrium
    numEqulibriumAttacker = equlibrium
print("оптимальная стратегия оброняющегося номер ", numEqulibriumDefender, "оптимальная стратегия атакующего номер ", numEqulibriumAttacker)

#arrayEqlibriumDefender.append(numEqulibriumDefender)
#arrayEqlibriumAttecker.append(numEqulibriumAttacker)

#print('array Defender - ', '\n', arrayEqlibriumDefender)
#print('arrau Attacker - ', '\n', arrayEqlibriumAttecker)

NameError: name 'np' is not defined